Here we are going to explore langchain parsers. These take the output from the llm and turn it into structured output eg. JSON

### Message

In [1]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

### Define the Schema

In [2]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


In [3]:

gift_schema = ResponseSchema(name="gift", description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days", description="How many days did it take for the product to arrive? If this information is not found, output -1.")
price_value_schema = ResponseSchema(name="price_value", description="Extract any sentences about the value or price, and output them as a comma-separated Python list.")

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

### Create an Output Parser

In [4]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

### Prompt Template

In [5]:
review_template = """
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.
price_value: Extract any sentences about the value or price, and output them as a comma-separated Python list.

text: {text}

{format_instructions}
"""

### Create Messages with Your Prompt

#### Tips for Prompt Templating
A template should contain keywords that lay out the chain-of-thought reasoning. eg.

Thought
Action
Obervation


In [6]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template=review_template)
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

### Call the API

In [9]:
import google.generativeai as palm
from langchain.chat_models.google_palm import ChatGooglePalm

In [15]:
import os
api_key = os.getenv("PALM_API_KEY")

In [16]:
chat = ChatGooglePalm(google_api_key=api_key, temperature=0.0)

In [19]:
response = chat(messages)

In [ ]:
messages

[HumanMessage(content='\nFor the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\nprice_value: Extract any sentences about the value or price, and output them as a comma-separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the 

In [20]:
output_dict = output_parser.parse(response.content)

In [21]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': ['slightly more expensive than the other leaf blowers out there']}